In [1]:
!git clone https://github.com/timothycao/agnews-classifier.git
%cd agnews-classifier

Cloning into 'agnews-classifier'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 37 (delta 17), reused 28 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 10.68 KiB | 3.56 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/kaggle/working/agnews-classifier


In [2]:
# Model

from peft import LoraConfig
from model import create_lora_model

lora_config = LoraConfig(
    r=5,
    lora_alpha=10,
    lora_dropout=0.1,
    bias='none',
    target_modules=['query', 'value', 'key', 'attention.output.dense'],
    task_type='SEQ_CLS'
)

model = create_lora_model(lora_config)

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 962,308 || all params: 125,611,016 || trainable%: 0.7661


In [4]:
# Training

from transformers import TrainingArguments
from train import main as train

training_args = TrainingArguments(
    # Core training configs
    max_steps=1000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    optim='adamw_torch',
    lr_scheduler_type='linear',
    learning_rate=5e-5,

    # Logging, evaluation, and checkpointing
    logging_strategy='steps',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    output_dir='/kaggle/working/saved_models',
    save_strategy='steps',
    save_steps=100,

    # Miscellaneous
    report_to='none',
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True}
)

train(model, training_args)

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Starting training...


Step,Training Loss,Validation Loss,Accuracy
100,1.352400,1.275176,0.845313
200,0.739500,0.371766,0.868750
300,0.370900,0.353653,0.887500
400,0.291700,0.346393,0.892188
500,0.317600,0.334053,0.893750
600,0.294500,0.329067,0.895312
700,0.308800,0.330115,0.898438
800,0.321000,0.314156,0.904687
900,0.303100,0.318972,0.903125
1000,0.293400,0.317138,0.898438


In [5]:
# Continue training for another 1000 steps

training_args = TrainingArguments(
    # Core training configs
    max_steps=2000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    optim='adamw_torch',
    lr_scheduler_type='linear',
    learning_rate=5e-5,

    # Logging, evaluation, and checkpointing
    logging_strategy='steps',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    output_dir='/kaggle/working/saved_models',
    save_strategy='steps',
    save_steps=100,

    # Miscellaneous
    report_to='none',
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True}
)

train(model, training_args, checkpoint='/kaggle/working/saved_models/checkpoint-1000')

Starting training...
Resuming from checkpoint: /kaggle/working/saved_models/checkpoint-1000


/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3418: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpoint, OPTIMIZER_NAME), map_

Step,Training Loss,Validation Loss,Accuracy
1100,0.299200,0.311523,0.904687
1200,0.305800,0.316786,0.907813
1300,0.333100,0.318645,0.901563
1400,0.291500,0.314598,0.906250
1500,0.298100,0.317339,0.898438
1600,0.326700,0.312690,0.901563
1700,0.275300,0.318920,0.907813
1800,0.293300,0.314307,0.904687
1900,0.269500,0.311942,0.904687
2000,0.338500,0.311090,0.903125


In [6]:
# Continue training for another 1000 steps

training_args = TrainingArguments(
    # Core training configs
    max_steps=3000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    optim='adamw_torch',
    lr_scheduler_type='linear',
    learning_rate=5e-5,

    # Logging, evaluation, and checkpointing
    logging_strategy='steps',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    output_dir='/kaggle/working/saved_models',
    save_strategy='steps',
    save_steps=100,

    # Miscellaneous
    report_to='none',
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True}
)

train(model, training_args, checkpoint='/kaggle/working/saved_models/checkpoint-2000')

Starting training...
Resuming from checkpoint: /kaggle/working/saved_models/checkpoint-2000


/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3418: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpoint, OPTIMIZER_NAME), map_

Step,Training Loss,Validation Loss,Accuracy
2100,0.303500,0.312155,0.907813
2200,0.302600,0.309273,0.906250
2300,0.277500,0.306150,0.901563
2400,0.306700,0.306209,0.904687
2500,0.277800,0.304118,0.907813
2600,0.271900,0.305659,0.907813
2700,0.260000,0.309768,0.909375
2800,0.271900,0.303490,0.910937
2900,0.275100,0.306296,0.906250
3000,0.279500,0.305592,0.906250


In [7]:
# Show all checkpoint metrics (including spread and ratio)

import pandas as pd

csv_path = '/kaggle/working/saved_models/checkpoint-3000/processed_log_history.csv'

df = pd.read_csv(csv_path)
df

,Step,Train Loss,Test Loss,Train Acc,Test Acc,Learning Rate,Epochs,Loss Spread,Loss Ratio,Acc Spread,Acc Ratio
0,100,1.3524,1.275176,0.426250,0.845313,0.000045,0.013405,0.077224,1.060560,-0.419063,0.504251
1,200,0.7395,0.371766,0.841518,0.868750,0.000040,0.026810,0.367734,1.989156,-0.027232,0.968654
2,300,0.3709,0.353653,0.872321,0.887500,0.000035,0.040214,0.017247,1.048769,-0.015179,0.982897
3,400,0.2917,0.346393,0.898214,0.892188,0.000030,0.053619,-0.054693,0.842107,0.006027,1.006755
4,500,0.3176,0.334053,0.892857,0.893750,0.000025,0.067024,-0.016453,0.950746,-0.000893,0.999001
5,600,0.2945,0.329067,0.896875,0.895312,0.000020,0.080429,-0.034567,0.894955,0.001563,1.001745
6,700,0.3088,0.330115,0.898214,0.898438,0.000015,0.093834,-0.021315,0.935432,-0.000223,0.999752
7,800,0.3210,0.314156,0.889732,0.904687,0.000010,0.107239,0.006844,1.021784,-0.014955,0.983469
8,900,0.3031,0.318972,0.900000,0.903125,0.000005,0.120643,-0.015872,0.950240,-0.003125,0.996540
9,1000,0.2934,0.317138,0.901786,0.898438,0.000000,0.134048,-0.023738,0.925149,0.003348,1.003727


In [8]:
# Inference

from inference import main as inference

data_path = '/kaggle/input/deep-learning-spring-2025-project-2/test_unlabelled.pkl'
checkpoint = '/kaggle/working/saved_models/checkpoint-2800'
output_dir = '/kaggle/working/saved_predictions'

inference(data_path, checkpoint, output_dir)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running inference...


100%|██████████| 250/250 [01:00<00:00,  4.16it/s]

Predictions saved to /kaggle/working/saved_predictions/predictions_checkpoint-2800.csv
